In [1]:
import pandas as pd
from pymatgen.io.cif import CifParser
from pyxtal import pyxtal
from pymatgen.symmetry.analyzer import SpacegroupAnalyzer
import os
from pathlib import Path
import zipfile

In [2]:
# file_path = "/home/holywater2/crystal_gen/mattergen/results/sg225_g1e3"
# file_path = "/home/holywater2/crystal_gen/mattergen/results/spg123_base"
file_path = "/home/holywater2/crystal_gen/mattergen/results/"
# file_path += "/spg38_gs0.00001_ge0.001"
file_path += "spg123_gs0.05_ge0.2_v4"

In [3]:
file_path = "/home/holywater2/crystal_gen/mattergen/results/prim/spg6_gs1_ge3_prim_fin/projected"

In [ ]:
# spg = file_path.split("/")[-1].split("_")[0].replace("spg", "")
# spg = int(spg)
# print(spg)

ValueError: invalid literal for int() with base 10: 'projected'

In [8]:
if not os.path.exists(file_path + "/generated_crystals_cif.zip"):
    print("No zip file found")
elif os.path.exists(file_path + "/generated_crystals_cif"):
    print("Directory already exists")
else:
    zipfile.ZipFile(file_path + "/generated_crystals_cif.zip", 'r').extractall(file_path)
    os.rename(file_path + "/tmp", file_path + "/generated_crystals_cif")
cif_path = file_path + "/generated_crystals_cif"
print(cif_path)

/home/holywater2/crystal_gen/mattergen/results/prim/spg123_gs1_ge3_prim_v5/generated_crystals_cif


In [9]:
# get list of cifs from directory
cif_list = [f for f in os.listdir(cif_path) if f.endswith('.cif')]
cif_list.sort(key=lambda x: int(x.split('_')[1].split('.')[0]))
cif_list = [os.path.join(cif_path, f) for f in cif_list]

In [10]:
structures = []
for cif_file in cif_list:
    parser = CifParser(cif_file)
    structure = parser.get_structures(primitive=False)[0]
    structures.append(structure)

/tmp/ipykernel_1285907/3946668647.py:4: FutureWarning: get_structures is deprecated; use parse_structures in pymatgen.io.cif instead.
The only difference is that primitive defaults to False in the new parse_structures method.So parse_structures(primitive=True) is equivalent to the old behavior of get_structures().
  structure = parser.get_structures(primitive=False)[0]


In [11]:
sga = SpacegroupAnalyzer(structure,symprec=0.1)

In [12]:
sga

In [13]:
res = {}
for i,structure in enumerate(structures):
    sgn = SpacegroupAnalyzer(structure,symprec=0.1).get_space_group_number()
    # print(i,sgn)
    if sgn not in res:
        res[sgn] = 0
    res[sgn] += 1
for k in sorted(res.keys()):
    print(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}")
print()

if spg in res:
    print(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}")
else:
    print(f"spg{spg} is not found")
    res[spg] = 0

  1 12  0.24
  2 4  0.08
  3 1  0.02
  8 3  0.06
 10 3  0.06
 12 4  0.08
 65 1  0.02
 99 1  0.02
119 1  0.02
123 6  0.12
139 1  0.02
162 1  0.02
166 3  0.06
198 1  0.02
216 1  0.02
221 3  0.06
225 3  0.06
229 1  0.02

123 6  0.12


In [14]:
res = {}
for i,structure in enumerate(structures):
    sgn = SpacegroupAnalyzer(structure,symprec=0.01).get_space_group_number()
    # print(i,sgn)
    if sgn not in res:
        res[sgn] = 0
    res[sgn] += 1
for k in sorted(res.keys()):
    print(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}")
print()

if spg in res:
    print(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}")
else:
    print(f"spg{spg} is not found")
    res[spg] = 0

  1 26  0.52
  2 1  0.02
  5 1  0.02
  6 1  0.02
  8 1  0.02
 10 1  0.02
 12 2  0.04
 25 1  0.02
 65 1  0.02
115 1  0.02
123 5  0.10
139 1  0.02
160 1  0.02
162 1  0.02
166 1  0.02
221 3  0.06
225 1  0.02
229 1  0.02

123 5  0.10


In [15]:
# write the number of spg
with open(file_path + f"/spg{spg}_{res[spg]}_{len(structures)}.txt", "w") as f:
    for k in sorted(res.keys()):
        f.write(f"{k:3} {res[k]}  {res[k]/len(structures):.2f}\n")
    f.write("\n")
    if spg in res:
        f.write(f"{spg} {res[spg]}  {res[spg]/len(structures):.2f}\n")
    else:
        f.write(f"spg{spg} is not found\n")